This notebook will be used for Capstone Project of Coursera 

In [12]:
import pandas as pd 

import numpy as np



print ("Hello Capstone Project Course !")

Hello Capstone Project Course !


In [13]:
import requests
import lxml.html as lh

In [14]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [15]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [16]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print(i,name)
    col.append((name,[]))

1 Postcode
2 Borough
3 Neighbourhood



In [17]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        
        i+=1

In [18]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [19]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [20]:
# Stripping the white spaces of column 
df.columns = df.columns.str.strip()
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [23]:
# Selecting only the rows which have borough assigned 

df_pro= df[df['Borough'] != 'Not assigned']

#df_pro= df.loc[df['Borough'] != 'Not assigned']


df_pro['Neighbourhood'] = df_pro['Neighbourhood'].str.rstrip('\n')
df_pro.head()

#df_pro.shape

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


**Now combining two rows  into one row with the neighborhoods separated with a comma**

In [24]:
df_pro_1 = df_pro.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))

df_pro_1.reset_index(level=['Postcode','Borough'], inplace=True)

df_pro_1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


**If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.**

In [27]:

   
    
    df_pro_1['Neighbourhood']=df_pro_1['Neighbourhood'].replace('Not assigned', df_pro_1['Borough'])
    
    df_pro_1.shape


(103, 3)